# EJERCICIOS

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Input, Flatten

2025-01-07 19:55:39.632879: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-07 19:55:39.636345: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-07 19:55:39.646240: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-07 19:55:39.667495: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-07 19:55:39.672172: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-07 19:55:39.684470: I tensorflow/core/platform/cpu_feature_gu

## EJERCICIO 1

¿Cuántas neuronas hacen falta en la capa de salida para clasificar emails entre correo no deseado y correo deseado? 

¿Qué función de activación deberías usar en la capa de salida? 

Si, en cambio, quieres enfrentarte a MNIST, ¿cuántas neuronas necesitas en la capa de salida y qué función de activación deberías usar? 

¿Y si quieres que la red prediga los precios de la vivienda en California?

- 1, es clasificación binaria.
- Depende.
    - En clasificación binaria se suele usar la fución de activación sigmoide.
    - En clasificación muticlase se suele usar la función de activación softmax.
    - En regrsión no se suele usar ninguna función de activación en específica.
- 10 neuronas de salida (una por numero escrito a mano 0-9) con función softmax. 
- Ninguna, no queremos que la neurona transforme el valor de salida.

## EJERCICIO 2

Entrena un red neuronal profunda con el conjunto de datos MNIST (puedes cargarlo con tf. keras. datasets.minst.load_data()). Intenta conseguir la mejor pruebas en el conjunto de validación  ajustando los hiperparámetros de modo manual. Prueba a buscar una buena tasa de aprendizaje y utiliza la detención temprana (mínimo hay que conseguir un 97%).

Pasos a seguir: 
+ Entrena varios modelos con distintas tasas de aprendizaje
+ Guarda cada modelo en un fichero
+ Carga cada modelo y evalúalo en el conjunto de cargas y quedate con programación con el mejor (bucle que se quede con el mejor), mostrando la tasa de aprendizaje y la exactitud del mejor modelo.

Si no obtienes más de un 97% vuelve a empezar.

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalización de las imágenes (opcional pero recomendado)
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# División del conjunto de entrenamiento en train y validation
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=17)

# Imprimir tamaños de los conjuntos
print(f"Conjunto de entrenamiento: {x_train.shape}")
print(f"Conjunto de validación: {x_val.shape}")
print(f"Conjunto de prueba: {x_test.shape}")


Conjunto de entrenamiento: (48000, 28, 28)
Conjunto de validación: (12000, 28, 28)
Conjunto de prueba: (10000, 28, 28)


In [12]:
learning_rates=[7e-4,1e-3,5e-3]
models=[]
histories=[]

In [13]:
tf.keras.backend.clear_session()

tf.random.set_seed(17)
i=0
for lr in learning_rates:
    
    model = Sequential(
        [
            Input(shape=(28, 28)),
            Flatten(),
            Dense(126, activation="relu"),
            Dense(42, activation="relu"),
            Dense(10, activation="softmax"),
        ]
)
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rates[i])
    

    i+=1
    model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])
    
    

    models.append(model)


In [14]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    patience=10, restore_best_weights=True
)

In [15]:
for model in models:
    print("Learning rate:", model.optimizer.learning_rate.numpy())
    histories.append(model.fit(x_train, y_train, epochs=30, validation_data=(x_val, y_val),callbacks=early_stopping_cb))


Learning rate: 0.0007
Epoch 1/30
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8542 - loss: 0.5245 - val_accuracy: 0.9514 - val_loss: 0.1705
Epoch 2/30
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9574 - loss: 0.1461 - val_accuracy: 0.9644 - val_loss: 0.1257
Epoch 3/30
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9703 - loss: 0.0988 - val_accuracy: 0.9703 - val_loss: 0.1075
Epoch 4/30
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9780 - loss: 0.0725 - val_accuracy: 0.9712 - val_loss: 0.1044
Epoch 5/30
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9845 - loss: 0.0550 - val_accuracy: 0.9725 - val_loss: 0.1030
Epoch 6/30
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9883 - loss: 0.0412 - val_accuracy: 0.9733 - val_loss: 0.1044
Epoch 7/30
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9911 - loss: 0.0310 - val_accuracy: 0.9728 - val_loss: 0.1070
Epoch 8/30
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.

In [16]:
for idx, history in enumerate(histories, start=1):
    print(f"\nResultados finales del modelo {idx}:")
    print("Learning rate:", models[idx-1].optimizer.learning_rate.numpy())
    
    # Obtener la última epoch registrada
    final_epoch = len(history.history['loss'])
    
    # Imprimir la última pérdida y precisión
    print(f"Última pérdida: {history.history['loss'][-1]}")
    print(f"Última precisión en entrenamiento: {history.history['accuracy'][-1]}")
    print(f"Última pérdida en validación: {history.history['val_loss'][-1]}")
    print(f"Última precisión en validación: {history.history['val_accuracy'][-1]}")


Resultados finales del modelo 1:
Learning rate: 0.0007
Última pérdida: 0.01042446494102478
Última precisión en entrenamiento: 0.9966250061988831
Última pérdida en validación: 0.15090486407279968
Última precisión en validación: 0.9731666445732117

Resultados finales del modelo 2:
Learning rate: 0.001
Última pérdida: 0.017509009689092636
Última precisión en entrenamiento: 0.9937499761581421
Última pérdida en validación: 0.14053259789943695
Última precisión en validación: 0.9736666679382324

Resultados finales del modelo 3:
Learning rate: 0.005
Última pérdida: 0.05629468709230423
Última precisión en entrenamiento: 0.9844375252723694
Última pérdida en validación: 0.16433031857013702
Última precisión en validación: 0.9710000157356262


In [17]:
models[0].save("resultados4_3/my_keras_model_0.keras")
models[1].save("resultados4_3/my_keras_model_1.keras")
models[2].save("resultados4_3/my_keras_model_2.keras")

In [18]:
modelo1 = tf.keras.models.load_model("resultados4_3/my_keras_model_0.keras")
modelo2 = tf.keras.models.load_model("resultados4_3/my_keras_model_1.keras")
modelo3 = tf.keras.models.load_model("resultados4_3/my_keras_model_2.keras")

modelos=[modelo1,modelo2,modelo3]

In [19]:
for modelo in modelos:
    test_loss, test_accuracy = modelo.evaluate(x_test, y_test)
    print("Conjunto de prueba - Pérdida: ",test_loss, " Precisión: ",test_accuracy)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 974us/step - accuracy: 0.9705 - loss: 0.1020
Conjunto de prueba - Pérdida:  0.08782917261123657  Precisión:  0.9735999703407288
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 990us/step - accuracy: 0.9490 - loss: 0.1651
Conjunto de prueba - Pérdida:  0.1442219763994217  Precisión:  0.9544000029563904
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 936us/step - accuracy: 0.9527 - loss: 0.1546
Conjunto de prueba - Pérdida:  0.1348491758108139  Precisión:  0.9581999778747559


## EJERCICIO 3

Clasificación de Imágenes de Dígitos Escritos a Mano:

* Objetivo: Entrenar una red neuronal para clasificar imágenes de dígitos escritos a mano utilizando el conjunto de datos SVHN (Street View House Numbers).

* Tareas: 
    * Cargar el conjunto de datos, dividir los datos (train, valid y test), explorar los datos representando alguna imagen con su etiqueta y muestra los valores (únicos) de y (en el entrenamiento, en validación y en pruebas), preparar los datos (d normalizar dividiendo entre 255.0 e igual hay que hacer algo más), construir, compilar y entrenar una red neuronal. 

    * Utilizar detección temprana

    * Hacer una gráfica de como evoluciona accuracy y val_accuracy durante el entrenamiento

    * Si es necesario ajustar el modelo para lograr una mayor precisión.

    * Evalúar el modelo en el conjunto de test y hacer alguna predicción (representa alguna imagen con su etiqueta real y la predicha)

In [51]:
def load_svhn_data(url):
    path = tf.keras.utils.get_file("svhn.mat", url)
    data = loadmat(path)
    return np.transpose(data["X"], (3, 0, 1, 2)), data["y"].flatten()

# Cargar datos
train_url = "http://ufldl.stanford.edu/housenumbers/train_32x32.mat"
test_url = "http://ufldl.stanford.edu/housenumbers/test_32x32.mat"
X_train_full, y_train_full = load_svhn_data(train_url)
X_test, y_test = load_svhn_data(test_url)

182040794/182040794 ━━━━━━━━━━━━━━━━━━━━ 26s 0us/step


## EJERCICIO 4

Predicción del consumo de energía:

* Objetivo: Construir una red neuronal para predecir el consumo de energía de edificios a partir de características como la temperatura, humedad, y tiempo del conjunto de datos Energy Efficiency de UCI Machine Learning Repository

* Tareas: 
    * Cargar el conjunto de datos, dividir los datos (train, valid y test), explorar los datos, preparar los datos (normalizar con una capa de normalización), construir un modelo de regresión, compilar y entrenar una red neuronal. 
    
    * Usa X1-X8 para la X e Y1-Y2 para la y

    * Ajusta el modelo hasta encontrar un error absoluto medio aceptable

    * Haz una gráfica de como evoluciona el error absoluto medio y error absoluto medio del conjunto de validación durante el entrenamiento

    * Evalúa el modelo en el conjunto de test y haz alguna predicción (muestra el valor predicho y el real)

In [16]:
# Descargar el conjunto de datos de eficiencia energética de UCI
url = ("https://archive.ics.uci.edu/ml/machine-learning-databases/00242/ENB2012_data.xlsx")
data = pd.read_excel(url)